## Ingesting PDFs

In [8]:
%pip install unstructured langchain
%pip install "unstructured[all-docs]"

Note: you may need to restart the kernel to use updated packages.

  Using cached xlrd-2.0.1-py2.py3-none-any.whl.metadata (3.4 kB)
  Using cached pdfminer.six-20240706-py3-none-any.whl.metadata (4.1 kB)
  Using cached pikepdf-9.2.0-cp311-cp311-win_amd64.whl.metadata (8.3 kB)
  Using cached pdf2image-1.17.0-py3-none-any.whl.metadata (6.2 kB)
  Using cached pi_heif-0.18.0-cp311-cp311-win_amd64.whl.metadata (6.8 kB)
  Using cached google_cloud_vision-3.7.4-py2.py3-none-any.whl.metadata (5.2 kB)
  Using cached python_oxmsg-0.0.1-py3-none-any.whl.metadata (5.0 kB)
  Using cached python_docx-1.1.2-py3-none-any.whl.metadata (2.0 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached unstructured.pytesseract-0.3.13-py3-none-any.whl.metadata (11 kB)
  Using cached pypandoc-1.13-py3-none-any.whl.metadata (16 kB)
  Using cached python_pptx-1.0.2-py3-none-any.whl.metadata (2.5 kB)
  Using cached unstructured_inference-0.7.36-py3-none-any.whl.metadata (5.9 kB)
  Using ca

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-proto 1.26.0 requires protobuf<5.0,>=3.19, but you have protobuf 5.28.0 which is incompatible.
tensorboard 2.15.1 requires protobuf<4.24,>=3.19.6, but you have protobuf 5.28.0 which is incompatible.
tensorflow-intel 2.15.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.28.0 which is incompatible.


In [9]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader

In [13]:
import os
os.getcwd()

'd:\\VISUAL_STUDIO\\LangChain\\Updated-Langchain-main\\rag'

In [43]:
from langchain_community.document_loaders import PyPDFLoader
local_path = "WEF_The_Global_Cooperation_Barometer_2024.pdf"

loader = PyPDFLoader(file_path=local_path)
data = loader.load()


In [44]:
data[1].page_content

'Contents\nImages: Getty Images\n© 2024 World Economic Forum. All rights \nreserved. No part of this publication may \nbe reproduced or transmitted in any form \nor by any means, including photocopying \nand recording, or by any information \nstorage and retrieval system.Disclaimer \nThis document is published by the  \nWorld Economic Forum as a contribution \nto a project, insight area or interaction. \nThe findings, interpretations and \nconclusions expressed herein are a result \nof a collaborative process facilitated and \nendorsed by the World Economic Forum \nbut whose results do not necessarily \nrepresent the views of the World Economic \nForum, nor the entirety of its Members, \nPartners or other stakeholders.Foreword \nAbout the Global Cooperation Barometer\nExecutive summary \nIntroduction: The state of global cooperation\nFive pillars of global cooperation\nPillar 1 Trade and capital \nPillar 2 Innovation and technology\nPillar 3 Climate and natural capital\nPillar 4 Health

## Vector Embeddings

In [45]:
!ollama pull nomic-embed-text

pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠸ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                         
verifying sha256 digest 
writing manifest 
success 


In [46]:
!ollama list

NAME                   	ID          	SIZE  	MODIFIED       
nomic-embed-text:latest	0a109f422b47	274 MB	13 seconds ago	
llama2:latest          	78e26419b446	3.8 GB	3 weeks ago   	
llama3:latest          	365c0bd3c000	4.7 GB	6 weeks ago   	


In [47]:
%pip install chromadb
%pip install langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [48]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [49]:
# Split and chunk
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 7500, chunk_overlap = 100)
chunks = text_splitter.split_documents(data)

In [50]:
# Add to Vector Database
vector_db = Chroma.from_documents(
    documents = chunks, 
    embedding = OllamaEmbeddings(model = "nomic-embed-text", show_progress = True),
    collection_name = "local-rag"
    )

OllamaEmbeddings: 100%|██████████| 26/26 [01:13<00:00,  2.83s/it]


## Retrieval

In [51]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [53]:
local_model = "llama3"
llm = ChatOllama(model = local_model)

In [54]:
QUERY_PROMPT = PromptTemplate(
    input_variables = ["question"],
    template = """You are an AI language model assistant. Your task is to generate five different versions of
    the given user question to retrieve relevant documents from a vector database. By generating multiple 
    perspectives on the user question, your goal is to help the user overcome some of the limitations of the
    distance-based similarity search. Provide these alternative questions seperated by new lines.
    Original question: {question}""",
)

In [56]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(),
    llm,
    prompt = QUERY_PROMPT
)

# RAG PROMPT
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [58]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [59]:
chain.invoke(input(""))

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.19s/it]


'Based on the provided context, this appears to be an "Insight Report" titled "The Global Cooperation Barometer 2024" published by the World Economic Forum (WEF) in collaboration with McKinsey & Company. The report focuses on measuring global cooperation across five dimensions: trade and capital, innovation and technology, climate and natural capital, health and wellness, and peace and security.'